In [21]:
import pandas as pd
import numpy as np
import torch

In [22]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/My Drive/FintechDS

In [23]:
df = pd.read_csv("data/transformed/df.csv",index_col=False)
reviews_scraped = pd.read_csv("data/transformed/reviews_scraped.csv",index_col=False)
products_scraped = pd.read_csv("data/transformed/products_scraped.csv",index_col=False)

In [24]:
reviews_scraped.dropna(inplace= True)
products_scraped.dropna(inplace= True)

In [25]:
df.review_text = df.review_text.astype(str)
df.review_title = df.review_title.astype(str)
reviews_scraped.review_text = reviews_scraped.review_text.astype(str)

In [26]:
df.head(2)

,Unnamed: 0,rating,verified,PRODUCT_CATEGORY,product_id,review_title,review_text,cat_0,cat_1,cat_2,...,text_sentiment,text_subjectivity,rating_count,rating_avg,rating1,rating2,rating3,rating4,rating5,product_title
0,0,4.0,0.0,PC,B00008NG7N,useful,"When least you think so, this product will sav...",0.0,0.0,0.0,...,0.200000,0.40000,233.0,4.6,0.03,0.01,0.07,0.07,0.81,"Targus PAUK10U Ultra Mini USB Keypad, Black"
1,1,4.0,1.0,Wireless,B00LH0Y3NM,New era for batteries,Lithium batteries are something new introduced...,0.0,0.0,0.0,...,0.719226,0.38532,214.0,3.8,0.17,0.06,0.14,0.11,0.53,Note 3 Battery : Stalion Strength Replacement ...


In [27]:
reviews_scraped.head(2)

,Unnamed: 0,product_id,review_title,rating_count,rating_avg,rating1,rating2,rating3,rating4,rating5,helpful,verified,review_text,rating,text_sentiment,text_subjectivity
0,0,0006486118,"A Dance With Dragons (A Song of Ice and Fire, ...",17539.0,4.5,0.03,0.03,0.07,0.14,0.73,6,1.0,"To preface this review, I wanted to inform you...",5,0.124998,0.521898
1,1,0006486118,"A Dance With Dragons (A Song of Ice and Fire, ...",17539.0,4.5,0.03,0.03,0.07,0.14,0.73,13,1.0,Fans have been looking forward to this one for...,4,0.107453,0.520954


In [28]:
#norm

from sklearn.preprocessing import StandardScaler

cols_to_norm = ['text_sentiment', 	'text_subjectivity', 	'rating_count', 	'rating_avg', 	'rating1', 	'rating2' ,	'rating3' ,	'rating4', 	'rating5']

df[cols_to_norm]=StandardScaler().fit_transform(df[cols_to_norm])


In [29]:
cols_to_norm = ['helpful', 'text_sentiment', 	'text_subjectivity', 	'rating_count', 	'rating_avg', 	'rating1', 	'rating2' ,	'rating3' ,	'rating4', 	'rating5']
reviews_scraped[cols_to_norm]=StandardScaler().fit_transform(reviews_scraped[cols_to_norm])

In [30]:
cat_cols = [ x for x in df.columns if 'cat_' in x]
#rating_cols = [ x for x in df.columns if 'rating' in x]
rating_cols = ['rating_count', 	'rating_avg','rating1', 'rating2', 'rating3', 'rating4', 'rating5']

In [31]:
reviews_scraped.shape

(99098, 16)

In [32]:
products_scraped.head(2)

,Unnamed: 0,product_id,product_title,rating_count,rating_avg,rating1,rating2,rating3,rating4,rating5
0,0,0006486118,"A Dance With Dragons (A Song of Ice and Fire, ...",17539.0,4.5,0.03,0.03,0.07,0.14,0.73
1,1,0028703200,Research in Music Education: An Introduction t...,7.0,4.8,0.00,0.00,0.00,0.23,0.77


In [33]:
reviews_scraped = pd.merge(products_scraped[['product_id','product_title']], reviews_scraped, on = 'product_id')

In [34]:
#device = 'cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data split

In [35]:
from sklearn.model_selection import train_test_split

x = df.loc[:, df.columns != 'label']
y = df.label

x_train, x_val, y_train, y_val = train_test_split( x, y, test_size=0.2,shuffle=True, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5,shuffle=True, random_state=1)

In [36]:
reviews_scraped_train = pd.merge(x_train.product_id, reviews_scraped, on = 'product_id')
reviews_scraped_val = pd.merge(x_val.product_id, reviews_scraped, on = 'product_id')
reviews_scraped_test = pd.merge(x_test.product_id, reviews_scraped, on = 'product_id')


In [37]:
#Rating_Purchase_Subjectivity

rps_cols = ['rating_count','rating_avg','rating1',  'rating2', 'rating3', 'rating4', 'rating5', 'verified', 'text_sentiment', 'text_subjectivity']

x_train[rps_cols]


,rating_count,rating_avg,rating1,rating2,rating3,rating4,rating5,verified,text_sentiment,text_subjectivity
1956,-0.156542,-0.556522,0.849689,-0.079519,0.213849,-0.682350,-0.063856,0.0,-1.551357,-1.484614
5304,-0.322242,0.248591,-0.625143,0.081413,0.083762,0.572927,-0.009298,0.0,-0.685249,-0.292011
16119,-0.332059,-1.764192,1.095495,1.529800,1.254546,0.447399,-1.700574,1.0,-0.505861,-0.385260
9226,-0.252561,0.248591,-0.256435,-0.240451,-0.046325,0.321871,0.099816,1.0,-0.381209,0.721356
4195,-0.299494,-0.556522,0.972592,0.081413,-0.176412,-0.682350,-0.063856,1.0,-0.159359,0.182566
...,...,...,...,...,...,...,...,...,...,...
10955,1.464901,1.053705,-0.748045,-0.723247,-0.566673,-0.431295,1.081848,1.0,0.610645,0.763504
17289,-0.329904,-0.153965,0.112273,0.081413,0.083762,0.321871,-0.227527,1.0,0.071642,0.919142
5192,-0.337088,-0.757801,-0.993851,3.782848,-1.217109,2.079258,-1.155002,0.0,0.434571,-0.407031
12172,-0.337567,-0.153965,-0.993851,-0.884179,2.815592,2.330314,-1.427788,0.0,0.747642,1.437939


# BERT

In [38]:
#%pip install sentence_transformers

In [39]:
import transformers
from transformers import BertTokenizerFast,AutoTokenizer, BertTokenizer, AutoModel
#importing automodel didn't work with VSCode
pretrained_weights = 'bert-base-uncased'
# bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Text Embeddings

In [40]:
from tqdm.notebook import tqdm_notebook
def prepareDataForBert(data):
      tokenized = []

      pbar = tqdm_notebook(total = len(data))
      pbar.set_description(f'Tokenized: ')
      for i in data:
        tokenized.append(tokenizer.encode(i, add_special_tokens = True, truncation=True))
        pbar.update(1)
      pbar.close()
    
      max_len = 0
      for i in tokenized:
        if len(i) > max_len:
          max_len = len(i)

      padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized])
      attention_mask = np.where(padded != 0, 1, 0)
      
      return padded, attention_mask

In [41]:
# idreviews_scraped = reviews_scraped['product_id'].unique()
# data = []
# for id in idreviews_scraped:
#     reviews = reviews_scraped.loc[reviews_scraped['product_id'] == id].values
#     data.append(reviews)

# data = np.asarray(data)

In [42]:
# new_product_reviews_scraped = pd.DataFrame({'id': idreviews_scraped, 'product_reviews_scraped':data})
# new_product_reviews_scraped.head(2)

# Scrapped review embeddings

In [43]:
#reviews_scrapped_ids, reviews_scrapped_mask
reviews_scrapped_ids_train, reviews_scrapped_mask_train = prepareDataForBert(reviews_scraped_train.review_text)
reviews_scrapped_ids_val,   reviews_scrapped_mask_val = prepareDataForBert(reviews_scraped_val.review_text)
reviews_scrapped_ids_test,  reviews_scrapped_mask_test = prepareDataForBert(reviews_scraped_test.review_text)


#reviews_scrapped_title_ids, reviews_scrapped_title_mask
reviews_scrapped_title_ids_train, reviews_scrapped_title_mask_train = prepareDataForBert(reviews_scraped_train.review_title)
reviews_scrapped_title_ids_val,   reviews_scrapped_title_mask_val = prepareDataForBert(reviews_scraped_val.review_title)
reviews_scrapped_title_ids_test,  reviews_scrapped_title_mask_test = prepareDataForBert(reviews_scraped_test.review_title)




  0%|          | 0/87083 [00:00<?, ?it/s]

  0%|          | 0/10908 [00:00<?, ?it/s]

  0%|          | 0/10652 [00:00<?, ?it/s]

  0%|          | 0/87083 [00:00<?, ?it/s]

  0%|          | 0/10908 [00:00<?, ?it/s]

  0%|          | 0/10652 [00:00<?, ?it/s]

In [44]:
#Product Title[str, ...] (BERT) => product_title_ids, product_title_mask
df_Product_Title_scrapped_ids_train, df_Product_Title_scrapped_mask_train = prepareDataForBert(reviews_scraped_train.product_title)
df_Product_Title_scrapped_ids_val,   df_Product_Title_scrapped_mask_val= prepareDataForBert(reviews_scraped_val.product_title)
df_Product_Title_scrapped_ids_test,  df_Product_Title_scrapped_mask_test = prepareDataForBert(reviews_scraped_test.product_title)


  0%|          | 0/87083 [00:00<?, ?it/s]

  0%|          | 0/10908 [00:00<?, ?it/s]

  0%|          | 0/10652 [00:00<?, ?it/s]

# Normal Review embeddings (not scrapped)


In [45]:
df_Review_Text_ids_train, df_Review_Text_mask_train = prepareDataForBert(x_train.review_text)
df_Review_Text_ids_val,   df_Review_Text_mask_val= prepareDataForBert(x_val.review_text)
df_Review_Text_ids_test,  df_Review_Text_mask_test = prepareDataForBert(x_test.review_text) 

  0%|          | 0/14485 [00:00<?, ?it/s]

  0%|          | 0/1811 [00:00<?, ?it/s]

  0%|          | 0/1811 [00:00<?, ?it/s]

In [46]:
df_Review_Title_ids_train, df_Review_Title_mask_train = prepareDataForBert(x_train.review_title)
df_Review_Title_ids_val,   df_Review_Title_mask_val= prepareDataForBert(x_val.review_title)
df_Review_Title_ids_test,  df_Review_Title_mask_test = prepareDataForBert(x_test.review_title) 

  0%|          | 0/14485 [00:00<?, ?it/s]

  0%|          | 0/1811 [00:00<?, ?it/s]

  0%|          | 0/1811 [00:00<?, ?it/s]

## Product Title embeddings (not scrapped)

In [47]:
#Product Title[str, ...] (BERT) => product_title_ids, product_title_mask
df_Product_Title_ids_train, df_Product_Title_mask_train = prepareDataForBert(x_train.product_title)
df_Product_Title_ids_val,   df_Product_Title_mask_val= prepareDataForBert(x_val.product_title)
df_Product_Title_ids_test,  df_Product_Title_mask_test = prepareDataForBert(x_test.product_title)


  0%|          | 0/14485 [00:00<?, ?it/s]

  0%|          | 0/1811 [00:00<?, ?it/s]

  0%|          | 0/1811 [00:00<?, ?it/s]

# Save embeddings

In [48]:
np.save('./data/embs/reviews_scrapped_ids_train.npy', reviews_scrapped_ids_train)
np.save('./data/embs/reviews_scrapped_mask_train.npy', reviews_scrapped_mask_train)
np.save('./data/embs/reviews_scrapped_ids_val.npy', reviews_scrapped_ids_val)
np.save('./data/embs/reviews_scrapped_mask_val.npy', reviews_scrapped_mask_val)
np.save('./data/embs/reviews_scrapped_ids_test.npy', reviews_scrapped_ids_test)
np.save('./data/embs/reviews_scrapped_mask_test.npy', reviews_scrapped_mask_test)
np.save('./data/embs/df_Review_Text_ids_train.npy', df_Review_Text_ids_train)
np.save('./data/embs/df_Review_Text_mask_train.npy', df_Review_Text_mask_train)
np.save('./data/embs/df_Review_Text_ids_val.npy', df_Review_Text_ids_val)
np.save('./data/embs/df_Review_Text_mask_val.npy', df_Review_Text_mask_val)
np.save('./data/embs/df_Review_Text_ids_test.npy', df_Review_Text_ids_test)
np.save('./data/embs/df_Review_Text_mask_test.npy', df_Review_Text_mask_test)
np.save('./data/embs/df_Review_Title_ids_train.npy', df_Review_Title_ids_train)
np.save('./data/embs/df_Review_Title_mask_train.npy', df_Review_Title_mask_train)
np.save('./data/embs/df_Review_Title_ids_val.npy', df_Review_Title_ids_val)
np.save('./data/embs/df_Review_Title_mask_val.npy', df_Review_Title_mask_val)
np.save('./data/embs/df_Review_Title_ids_test.npy', df_Review_Title_ids_test)
np.save('./data/embs/df_Review_Title_mask_test.npy', df_Review_Title_mask_test)
np.save('./data/embs/df_Product_Title_ids_train.npy', df_Product_Title_ids_train)
np.save('./data/embs/df_Product_Title_mask_train.npy', df_Product_Title_mask_train)
np.save('./data/embs/df_Product_Title_ids_val.npy', df_Product_Title_ids_val)
np.save('./data/embs/df_Product_Title_mask_val.npy', df_Product_Title_mask_val)
np.save('./data/embs/df_Product_Title_ids_test.npy', df_Product_Title_ids_test)
np.save('./data/embs/df_Product_Title_mask_test.npy', df_Product_Title_mask_test)


# Load saved embs

In [49]:
reviews_scrapped_ids_train = np.load('./data/embs/reviews_scrapped_ids_train.npy')
reviews_scrapped_mask_train = np.load('./data/embs/reviews_scrapped_mask_train.npy')
reviews_scrapped_ids_val = np.load('./data/embs/reviews_scrapped_ids_val.npy')
reviews_scrapped_mask_val = np.load('./data/embs/reviews_scrapped_mask_val.npy')
reviews_scrapped_ids_test = np.load('./data/embs/reviews_scrapped_ids_test.npy')
reviews_scrapped_mask_test = np.load('./data/embs/reviews_scrapped_mask_test.npy')
df_Review_Text_ids_train = np.load('./data/embs/df_Review_Text_ids_train.npy')
df_Review_Text_mask_train = np.load('./data/embs/df_Review_Text_mask_train.npy')
df_Review_Text_ids_val = np.load('./data/embs/df_Review_Text_ids_val.npy')
df_Review_Text_mask_val = np.load('./data/embs/df_Review_Text_mask_val.npy')
df_Review_Text_ids_test = np.load('./data/embs/df_Review_Text_ids_test.npy')
df_Review_Text_mask_test = np.load('./data/embs/df_Review_Text_mask_test.npy')
df_Review_Title_ids_train = np.load('./data/embs/df_Review_Title_ids_train.npy')
df_Review_Title_mask_train = np.load('./data/embs/df_Review_Title_mask_train.npy')
df_Review_Title_ids_val = np.load('./data/embs/df_Review_Title_ids_val.npy')
df_Review_Title_mask_val = np.load('./data/embs/df_Review_Title_mask_val.npy')
df_Review_Title_ids_test = np.load('./data/embs/df_Review_Title_ids_test.npy')
df_Review_Title_mask_test = np.load('./data/embs/df_Review_Title_mask_test.npy')


reviews_scrapped_title_ids_train = np.load('./data/embs/reviews_scrapped_ids_train.npy')
reviews_scrapped_title_mask_train = np.load('./data/embs/reviews_scrapped_mask_train.npy')
reviews_scrapped_title_ids_val = np.load('./data/embs/reviews_scrapped_ids_val.npy')  
reviews_scrapped_title_mask_val = np.load('./data/embs/reviews_scrapped_mask_val.npy')
reviews_scrapped_title_ids_test = np.load('./data/embs/reviews_scrapped_ids_test.npy')
reviews_scrapped_title_mask_test = np.load('./data/embs/reviews_scrapped_mask_test.npy')

df_Product_Title_ids_train = np.load('./data/embs/df_Product_Title_ids_train.npy')
df_Product_Title_mask_train = np.load('./data/embs/df_Product_Title_mask_train.npy')
df_Product_Title_ids_val = np.load('./data/embs/df_Product_Title_ids_val.npy')
df_Product_Title_mask_val = np.load('./data/embs/df_Product_Title_mask_val.npy')
df_Product_Title_ids_test = np.load('./data/embs/df_Product_Title_ids_test.npy')
df_Product_Title_mask_test = np.load('./data/embs/df_Product_Title_mask_test.npy')

# DEFINE MODEL 

In [50]:
import torch.nn as nn
class ReviewEmbNN(nn.Module):
    def __init__(self, freeze_bert=False):
      super(ReviewEmbNN, self).__init__()
       
      bertView_out = 768
      bertTitle_out = 768
      bertTitleReview_out = 768
      input_size = 10#4#36

      h0 = 1024
      h1 = 512
      h2 = 128
      h3 = 64
      # h4 = 32
      out = 1

      titleReviewIn = bertTitle_out + bertTitleReview_out + input_size
      titleReviewOut = bertTitle_out + bertTitleReview_out + input_size + 2
      concatenate = h3 + bertView_out
      concatenate1 = concatenate + 2


      self.bert = AutoModel.from_pretrained(pretrained_weights) # load pretrain model bert to get the embeddings 

      # self.bert_reviews = bert.from_pretrained(pretrained_weights)
      # self.prod_title =        bert.from_pretrained(pretrained_weights)
      # self.title_review = bert.from_pretrained(pretrained_weights)
      # self.reviews_scraped_text = self.bert_reviews
      # self.reviews_scrapped_title = self.title_review

      self.l1 = nn.Linear(titleReviewIn, titleReviewOut)
      self.act1 = nn.ReLU()
      self.d1 = nn.Dropout(p=0.5)

      self.l2 = nn.Linear(titleReviewOut, h0)
      self.act2 = nn.ReLU()
      self.d2 = nn.Dropout(p=0.5)

      self.l3 = nn.Linear(h0, h1)
      self.act3 = nn.ReLU()
      self.d3 = nn.Dropout(p=0.2)

      self.l4 = nn.Linear(h1, h1)
      self.act4 = nn.ReLU()
      self.d4 = nn.Dropout(p=0.2)

      self.l5 = nn.Linear(h1, h2)
      self.act5 = nn.ReLU()
      self.d5 = nn.Dropout(p=0.2)

      self.l6 = nn.Linear(h2, h3)
    #   self.act6 = nn.ReLU()
    #   self.d6 = nn.Dropout(p=0.2)

      # Freeze the BERT model
      if freeze_bert:
          for param in self.bert.parameters():
              param.requires_grad = False

          # for param in self.prod_title.parameters():
          #     param.requires_grad = False

          # for param in self.title_review.parameters():
          #     param.requires_grad = False
      

     

    #def forward(self, reviews, prod_title, title_review, s_reviews, s_title_review, inputs ):
    def forward(self, prod_title, title_review, s_reviews, s_title_review, inputs, helpfulness = None ):
        """"
        row_in_fake_real_products_df
            row_in_fake_real_products_df is a df with the following columns:
                - product_title
                - review_title
                - rating1, rating2, rating3, rating4, rating5 (the percentages of ratings from each brackets, as values between 0.0 and 1.0)
                - verified_purchase
                - sentiment, polarity
        

       
        Feed input to BERT and the classifier to compute logits.
        @param    reviews (obj) : ** keword argument [batch_size, hidden layer bert model : 512, output bert model 768]
        @param    inputs (torch.Tensor): Category(30) + Ranking(5) + Verified Purchased(1) 
        # migth have to change the size related to the inputs
        Our new inputs should be Rating(1) + Verified Purchase(1) + A Valence + Subjectivity(2) = 4 
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        

        # bert_reviews = self.bert_reviews(**s_reviews)
        # prod_title = self.prod_title(**prod_title) #product title
        # title_review = self.title_review(**title_review)

        s_title_review = self.bert(**s_title_review)
        s_reviews = self.bert(**s_reviews)



        #bert_reviews_state_cls = bert_reviews['last_hidden_state'][:, 0, :]
        #bert_prod_title_state_cls = prod_title['last_hidden_state'][:, 0, :]
        #bert_title_review_state_cls = title_review['last_hidden_state'][:, 0, :]

        s_title_review_state_cls = s_title_review['last_hidden_state'][:, 0, :]
        s_reviews_state_cls = s_reviews['last_hidden_state'][:, 0, :]


        #concatenate1 = torch.cat((bert_prod_title_state_cls, bert_title_review_state_cls, inputs), 1)
        concatenate1 = torch.cat(( s_title_review_state_cls, s_reviews_state_cls, inputs), 1)
        # print(inputs.shape)
        out = self.l1(concatenate1)
        out = self.act1(out)
        out = self.d1(out)

        out = self.l2(out)
        out = self.act2(out)
        out = self.d2(out)

        out = self.l3(out)
        out = self.act3(out)
        out = self.d3(out)

        out = self.l4(out)
        out = self.act4(out)
        out = self.d4(out)

        out = self.l5(out)
        out = self.act5(out)
        out = self.d5(out)

        out = self.l6(out)
        print(out.shape)
        


        #activate if helpulness is different from None
        if helpfulness is not None:
            print(helpfulness.shape)
            helpfulness = helpfulness.view(-1, 1)
            concatenate2 = torch.cat((out, helpfulness), 1)

        
        # out = self.act6(out)
        # out = self.d6(out)
        print(out.shape)
        #print(out)
        # return self.out(out)
        return out
        
        

# Create Torch Dataset

In [51]:
from torch.utils.data import Dataset, DataLoader

class DatasetClass(Dataset):
    ''' 
    @ Name Column  =>  @ Name Variable

    Review Embedding Network
        Review_Title (BERT)=>  df_Review_Title_ids, df_Review_Title_mask
        Review_Text (BERT)=>   df_Review_Text_ids, df_Review_Text_mask
        Rating, Verified Purchase, SA Valence, Subjectivity => Rating_Purchase_Subjectivity

    Black Box 
        Review(Scraped) (BERT)=> reviews_scrapped_ids, reviews_scrapped_mask
        Review_Title(Scraped) (BERT)=> reviews_scrapped_title_ids, reviews_scrapped_title_mask

    Product Title[str, ...] (BERT) => product_title_ids, product_title_mask

    Ratings, Ratio, RatingAvg, Rating Count => ratings

    Category (One hot encoding) => category 


    ----------------------------------------------------------------------------------------------------------------------------
        
    n_samples => Obtained from df dataframe col: PRODUCT_CATEGORY
    '''
    def __init__(
        self, 

        # Review Embedding Network
        df_Review_Title_ids, 
        df_Review_Title_mask,

        df_Review_Text_ids,
        df_Review_Text_mask,

        Rating_Purchase_Subjectivity,

        reviews_scrapped_ids,
        reviews_scrapped_mask,

        reviews_scrapped_title_ids, 
        reviews_scrapped_title_mask,

        product_title_ids,
        product_title_mask,

        ratings,

        category,
        helpful,

        ):

        self.df_Review_Title_ids =  torch.tensor(df_Review_Title_ids)
        self.df_Review_Title_mask =  torch.tensor(df_Review_Title_mask)

        self.df_Review_Text_ids =  torch.tensor(df_Review_Text_ids)
        self.df_Review_Text_mask =  torch.tensor(df_Review_Text_mask)

        self.reviews_scrapped_title_ids =  torch.tensor(reviews_scrapped_title_ids)
        self.reviews_scrapped_title_mask= torch.tensor(reviews_scrapped_title_mask)

        self.Rating_Purchase_Subjectivity = torch.from_numpy(np.asarray(Rating_Purchase_Subjectivity, dtype=np.float32))

        self.reviews_scrapped_ids =  torch.tensor(reviews_scrapped_ids)
        self.reviews_scrapped_mask =  torch.tensor(reviews_scrapped_mask)

        self.product_title_ids =  torch.tensor(product_title_ids)
        self.product_title_mask =  torch.tensor(product_title_mask)

        self.ratings = torch.from_numpy(np.asarray(ratings, dtype=np.float32))

        self.category = torch.from_numpy(np.asarray(category, dtype=np.int8))


        self.n_samples = len(category)

        self.helpful = torch.from_numpy(np.asarray(helpful, dtype=np.float32))

    def __getitem__(self, index):
        return self.df_Review_Title_ids[index], self.df_Review_Title_mask[index], \
            self.df_Review_Text_ids[index], self.df_Review_Text_mask[index], \
            self.Rating_Purchase_Subjectivity[index], \
            self.reviews_scrapped_ids[index], self.reviews_scrapped_mask[index], \
            self.reviews_scrapped_title_ids[index], self.reviews_scrapped_title_mask[index], \
            self.product_title_ids[index], self.product_title_mask[index], \
            self.ratings[index], \
            self.category[index], \
            self.helpful[index], \

    def __len__(self):
        return self.n_samples 

# Create dataset class objects

In [52]:
train = DatasetClass(
            df_Review_Title_ids_train, 
            df_Review_Title_mask_train,    
            df_Review_Text_ids_train,
            df_Review_Text_mask_train,    
            reviews_scraped_train[rps_cols], 
            reviews_scrapped_ids_train,
            reviews_scrapped_mask_train,
            reviews_scrapped_title_ids_train, 
            reviews_scrapped_title_mask_train,               
            df_Product_Title_ids_train,
            df_Product_Title_mask_train,    
            x_train[rating_cols],    # categorical columns
            x_train[cat_cols],
            reviews_scraped_train['helpful'],
            )

validate = DatasetClass(
            df_Review_Title_ids_val, 
            df_Review_Title_mask_val,    
            df_Review_Text_ids_val,
            df_Review_Text_mask_val,    
            reviews_scraped_val[rps_cols],    
            reviews_scrapped_ids_val,
            reviews_scrapped_mask_val,   
            reviews_scrapped_title_ids_val, 
            reviews_scrapped_title_mask_val,    
            df_Product_Title_ids_val,
            df_Product_Title_mask_val,    
            x_val[rating_cols],    # categorical columns
            x_val[cat_cols],
            reviews_scraped_val['helpful'],
            )

test = DatasetClass(
            df_Review_Title_ids_test, 
            df_Review_Title_mask_test,    
            df_Review_Text_ids_test,
            df_Review_Text_mask_test,    
            reviews_scraped_test[rps_cols],    
            reviews_scrapped_ids_test,
            reviews_scrapped_mask_test, 
            reviews_scrapped_title_ids_test, 
            reviews_scrapped_title_mask_test,      
            df_Product_Title_ids_test,
            df_Product_Title_mask_test,    
            x_test[rating_cols],    # categorical columns
            x_test[cat_cols],
            reviews_scraped_test['helpful'],
            )

In [53]:
from torch.utils.data import Dataset, DataLoader
batch_size = 8

train_dl = DataLoader(dataset = train, batch_size = batch_size, shuffle = True)
validation_dl = DataLoader(dataset = validate, batch_size = batch_size, shuffle = False)
test_dl = DataLoader(dataset = test, batch_size = batch_size, shuffle = False)


# DELETE VARS

In [54]:
import gc

if 'reviews_scrapped_ids_train' in globals(): del reviews_scrapped_ids_train
if 'reviews_scrapped_mask_train' in globals(): del reviews_scrapped_mask_train

if 'reviews_scrapped_ids_val' in globals(): del reviews_scrapped_ids_val
if 'reviews_scrapped_mask_val' in globals(): del reviews_scrapped_mask_val

if 'reviews_scrapped_ids_test' in globals(): del reviews_scrapped_ids_test
if 'reviews_scrapped_mask_test' in globals(): del reviews_scrapped_mask_test


if 'reviews_scrapped_title_ids_train' in globals(): del reviews_scrapped_title_ids_train
if 'reviews_scrapped_title_mask_train' in globals(): del reviews_scrapped_title_mask_train

if 'reviews_scrapped_title_ids_val' in globals(): del reviews_scrapped_title_ids_val
if 'reviews_scrapped_title_mask_val' in globals(): del reviews_scrapped_title_mask_val

if 'reviews_scrapped_title_ids_test' in globals(): del reviews_scrapped_title_ids_test
if 'reviews_scrapped_title_mask_test' in globals(): del reviews_scrapped_title_mask_test




if 'df_Product_Title_scrapped_ids_train' in globals(): del df_Product_Title_scrapped_ids_train
if 'df_Product_Title_scrapped_mask_train' in globals(): del df_Product_Title_scrapped_mask_train


if 'df_Product_Title_scrapped_ids_val' in globals(): del df_Product_Title_scrapped_ids_val
if 'df_Product_Title_scrapped_mask_val' in globals(): del df_Product_Title_scrapped_mask_val


if 'df_Product_Title_scrapped_ids_test' in globals(): del df_Product_Title_scrapped_ids_test
if 'df_Product_Title_scrapped_mask_test' in globals(): del df_Product_Title_scrapped_mask_test


if 'df_Review_Text_ids_train' in globals(): del df_Review_Text_ids_train
if 'df_Review_Text_mask_train' in globals(): del df_Review_Text_mask_train

if 'df_Review_Text_ids_val' in globals(): del df_Review_Text_ids_val
if 'df_Review_Text_mask_val' in globals(): del df_Review_Text_mask_val

if 'df_Review_Text_ids_test' in globals(): del df_Review_Text_ids_test
if 'df_Review_Text_mask_test' in globals(): del df_Review_Text_mask_test



if 'df_Review_Title_ids_train' in globals(): del df_Review_Title_ids_train
if 'df_Review_Title_mask_train' in globals(): del df_Review_Title_mask_train

if 'df_Review_Title_ids_val' in globals(): del df_Review_Title_ids_val
if 'df_Review_Title_mask_val' in globals(): del df_Review_Title_mask_val

if 'df_Review_Title_ids_test' in globals(): del df_Review_Title_ids_test
if 'df_Review_Title_mask_test' in globals(): del df_Review_Title_mask_test




if 'df_Product_Title_ids_train' in globals(): del df_Product_Title_ids_train
if 'df_Product_Title_mask_train' in globals(): del df_Product_Title_mask_train

if 'df_Product_Title_ids_val' in globals(): del df_Product_Title_ids_val
if 'df_Product_Title_mask_val' in globals(): del df_Product_Title_mask_val

if 'df_Product_Title_ids_test' in globals(): del df_Product_Title_ids_test
if 'df_Product_Title_mask_test' in globals(): del df_Product_Title_mask_test


if 'idreviews_scraped' in globals(): del idreviews_scraped
if 'data' in globals(): del data
if 'train' in globals(): del train
if 'validate' in globals(): del validate
if 'test' in globals(): del test
if 'tokenizer' in globals(): del tokenizer




gc.collect

<function gc.collect(generation=2)>

# Run Model

In [55]:
rev_emb_nn = ReviewEmbNN().to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [56]:
for idx, (df_Review_Title_ids, df_Review_Title_mask, df_Review_Text_ids, 
          df_Review_Text_mask, Rating_Purchase_Subjectivity, reviews_scrapped_ids, 
          reviews_scrapped_mask, reviews_scrapped_title_ids, reviews_scrapped_title_mask,
          product_title_ids, product_title_mask, ratings, category, helpful) in enumerate(train_dl):
 
  df_Review_Title = {
      "input_ids": df_Review_Title_ids.to(device), 
      "attention_mask": df_Review_Title_mask.to(device)
  }
  df_Review_Text ={
      "input_ids": df_Review_Text_ids.to(device), 
      "attention_mask": df_Review_Text_ids.to(device)
  }

  reviews_scrapped ={
     "input_ids": reviews_scrapped_ids.to(device), 
      "attention_mask": reviews_scrapped_mask.to(device) 
  }
  reviews_scrapped_title ={
     "input_ids": reviews_scrapped_title_ids.to(device), 
      "attention_mask": reviews_scrapped_title_mask.to(device) 
  }
  product_title ={
       "input_ids": product_title_ids.to(device), 
      "attention_mask": product_title_mask.to(device) 
  }
  Rating_Purchase_Subjectivity = Rating_Purchase_Subjectivity.to(device)
  ratings = ratings.to(device)
  category = category.to(device)
  helpful = helpful.to(device)


  emb64 = rev_emb_nn.forward(prod_title=df_Review_Title, title_review=df_Review_Title, s_reviews=reviews_scrapped, s_title_review= reviews_scrapped_title, inputs=Rating_Purchase_Subjectivity, helpfulness=helpful)

  break

torch.Size([8, 64])
torch.Size([8])
torch.Size([8, 64])


In [58]:
emb64.shape

torch.Size([8, 64])

# Add the output to an Encoder layer

